In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows',None)

In [3]:
netflix = pd.read_csv('/content/drive/MyDrive/combined_data_1.txt',header=None, names=['Cust_Id', 'Rating'], usecols=[0,1])

In [4]:
netflix.head()

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [5]:
len(netflix)

24058263

In [6]:
netflix.isnull().sum().sum()

4499

**Data Preprocessing :**

In [7]:
df_user = netflix.copy()

In [8]:
# Function to extract movie ID from Cust_Id
def extract_movie_id(cust_id):
    if ":" in cust_id:
        return int(cust_id.strip(":"))
    else:
        return None

# Extract movie IDs for rows where Rating is NaN
df_user['Movie_Id'] = df_user.loc[df_user['Rating'].isnull(), 'Cust_Id'].apply(extract_movie_id)

# Forward fill the extracted movie IDs to fill NaNs in other rows
df_user['Movie_Id'] = df_user['Movie_Id'].ffill()

# Drop rows with NaN ratings
df_user = df_user.dropna(subset=['Rating'])

# Convert Movie_Id to integer type
df_user['Movie_Id'] = df_user['Movie_Id'].astype(int)

# Optionally, if you want to reset the index after dropping rows
df_user.reset_index(drop=True, inplace=True)

In [9]:
df_user.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


In [10]:
# crosschecking

print('Number of unique movie IDs :', len(netflix) - len(df_user))

# which is equal to netflix.isnull().sum().sum()

Number of unique movie IDs : 4499


In [11]:
df_user['Movie_Id'].value_counts().sort_index()

1          547
2          145
3         2012
4          142
5         1140
6         1019
7           93
8        14910
9           95
10         249
11         198
12         546
13         125
14         118
15         290
16        2699
17        7108
18       10722
19         539
20         116
21         218
22         203
23         615
24        1333
25        1207
26        5861
27         273
28       39752
29         523
30      118413
31         221
32        1854
33        6890
34         108
35         839
36         939
37         281
38         802
39         752
40         491
41          93
42         128
43         105
44        8501
45        2619
46        6558
47        2350
48        3591
49         231
50         328
51          90
52        5147
53         114
54         159
55        2406
56        1833
57        3562
58       17405
59         204
60         295
61         147
62         290
63         184
64          95
65         258
66         166
67        

In [12]:
# Calculate count and mean of ratings for each movie
movie_summary = df_user.groupby('Movie_Id')['Rating'].agg(['count', 'mean'])

# Find the movie count threshold (taking top 70% rated movies)
movie_benchmark = round(movie_summary['count'].quantile(0.3), 0)

# Get the movie IDs with counts below the threshold
drop_movie_list = movie_summary[movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))
print('Number of movies to be dropped:', len(drop_movie_list))
print('List of movies to be dropped:')
print(drop_movie_list)

Movie minimum times of review: 227.0
Number of movies to be dropped: 1348
List of movies to be dropped:
Int64Index([   2,    4,    7,    9,   11,   13,   14,   20,   21,   22,
            ...
            4464, 4466, 4469, 4471, 4475, 4480, 4481, 4486, 4487, 4494],
           dtype='int64', name='Movie_Id', length=1348)


In [13]:
# Calculate count of ratings for each customer
cust_summary = df_user.groupby('Cust_Id')['Rating'].count()

# Find the customer count threshold (taking top 70% rated customers)
cust_benchmark = round(cust_summary.quantile(0.3), 0)

# Get the customer IDs with counts below the threshold
drop_cust_list = cust_summary[cust_summary < cust_benchmark].index

print('\nCustomer minimum times of review: {}'.format(cust_benchmark))
print('Number of customers to be dropped:', len(drop_cust_list))
print('List of customers to be dropped:')
print(drop_cust_list)


Customer minimum times of review: 11.0
Number of customers to be dropped: 140127
List of customers to be dropped:
Index(['1000004', '1000038', '1000051', '1000057', '1000122', '1000131',
       '1000145', '1000181', '1000194', '1000199',
       ...
       '999747', '999769', '999793', '999842', '999875', '999878', '999883',
       '999884', '999904', '99994'],
      dtype='object', name='Cust_Id', length=140127)


In [14]:
# Filter out the rows with movies in the drop list
df_user = df_user[~df_user['Movie_Id'].isin(drop_movie_list)]
# Filter out the rows with customers in the drop list
df_user = df_user[~df_user['Cust_Id'].isin(drop_cust_list)]

In [15]:
len(df_user)

23160492

In [16]:
# Number of unique movie IDs in the filtered DataFrame
unique_movie_ids = df_user['Movie_Id'].nunique()

print("Number of unique movie IDs after filtering :", unique_movie_ids)

Number of unique movie IDs after filtering : 3151


In [17]:
df_genre = pd.read_csv('/content/drive/MyDrive/movies.csv')

In [18]:
df_genre.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
len(df_genre)

27278

In [20]:
df_genre.rename(columns={'movieId': 'Movie_Id', 'title': 'Title', 'genres':'Genre'}, inplace=True)

In [21]:
df_genre.columns

Index(['Movie_Id', 'Title', 'Genre'], dtype='object')

In [22]:
df_genre.isnull().sum()

Movie_Id    0
Title       0
Genre       0
dtype: int64

In [23]:
df_genre.duplicated('Movie_Id').sum()

0

In [24]:
# Perform the inner join on 'Movie_Id'
df = pd.merge(df_user, df_genre, on='Movie_Id', how='inner')

In [25]:
df.head()

,Cust_Id,Rating,Movie_Id,Title,Genre
0,1488844,3.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,822109,5.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,885013,4.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,30878,4.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,823519,3.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [26]:
df.tail()

,Cust_Id,Rating,Movie_Id,Title,Genre
22928378,2591364,2.0,4499,Dirty Rotten Scoundrels (1988),Comedy
22928379,1791000,2.0,4499,Dirty Rotten Scoundrels (1988),Comedy
22928380,512536,5.0,4499,Dirty Rotten Scoundrels (1988),Comedy
22928381,988963,3.0,4499,Dirty Rotten Scoundrels (1988),Comedy
22928382,1704416,3.0,4499,Dirty Rotten Scoundrels (1988),Comedy


In [27]:
len(df)

22928383

In [28]:
df.duplicated(['Movie_Id','Cust_Id']).sum()

0

In [29]:
# Get unique Movie Ids from df_user
user_movie_ids = set(df_user['Movie_Id'].unique())

# Get unique Movie Ids from df_genre
genre_movie_ids = set(df_genre['Movie_Id'].unique())

# Find Movie Ids present in df_user but not in df_genre
missing_movie_ids = user_movie_ids - genre_movie_ids

if missing_movie_ids:
    print("Movie Ids present in df_user but not in df_genre:")
    print(missing_movie_ids)
else:
    print("All Movie Ids in df_user are present in df_genre.")

Movie Ids present in df_user but not in df_genre:
{1536, 3842, 1540, 646, 1800, 1418, 1802, 1803, 1530, 1048, 1560, 1786, 1435, 1691, 1818, 1182, 1823, 3482, 1700, 677, 1828, 3366, 1576, 4264, 811, 1195, 1451, 686, 1452, 1072, 689, 817, 1074, 1712, 1838, 2228, 1469, 578, 323, 1607, 1481, 1737, 3532, 1229, 1741, 1745, 978, 1491, 1108, 1492, 1494, 1618, 1751, 2645, 3416, 91, 863, 3935, 1505, 1506, 1634, 1761, 1637, 1638, 1766, 3815, 622, 1775, 1521, 1778, 883, 1402, 1403, 1790}


In [30]:
df.dtypes

Cust_Id      object
Rating      float64
Movie_Id      int64
Title        object
Genre        object
dtype: object

In [31]:
# Convert 'Cust_Id' to integer
df['Cust_Id'] = df['Cust_Id'].astype(int)

In [32]:
df.dtypes

Cust_Id       int64
Rating      float64
Movie_Id      int64
Title        object
Genre        object
dtype: object

In [33]:
df.to_csv('/content/drive/My Drive/netflix_filtered.csv', index=False)

**1. Find out the list of most popular and liked genre.**

In [34]:
# Create a copy of the dataframe to avoid changing the original dataframe
df_copy = df.copy()

# Create an empty list to store the split genres
split_genres = []

# Iterate through each row of the 'Genre' column
for genres in df_copy['Genre']:
    # Split the genres by '|'
    split_genres.extend(genres.split('|'))

# Get unique genres
unique_genres = np.unique(split_genres)

# Create an empty dataframe to store genre counts and mean ratings
genre_stats = pd.DataFrame(columns=['Genre', 'Total_Ratings', 'Average_Rating'])

# Iterate through unique genres to calculate counts and mean ratings
for genre in unique_genres:
    genre_ratings = df_copy[df_copy['Genre'].str.contains(genre, regex=False)]['Rating']
    total_ratings = len(genre_ratings)
    average_rating = genre_ratings.mean()
    genre_stats = genre_stats.append({'Genre': genre, 'Total_Ratings': total_ratings, 'Average_Rating': average_rating}, ignore_index=True)

# Sort genres by Total Ratings and Average Rating
most_popular_genres = genre_stats.sort_values(by=['Total_Ratings', 'Average_Rating'], ascending=False)

# Print the list of most popular and liked genres
print("List of most popular and liked genres:")
print(most_popular_genres)

List of most popular and liked genres:
          Genre Total_Ratings  Average_Rating
7         Drama      11046346        3.584694
4        Comedy       8129516        3.603046
14      Romance       3715504        3.608117
16     Thriller       3222878        3.547501
5         Crime       2830142        3.538527
0        Action       2691957        3.605900
10       Horror       2553700        3.735797
1     Adventure       2193805        3.683779
3      Children       1454306        3.663809
15       Sci-Fi       1423818        3.618338
8       Fantasy       1059801        3.643299
12      Musical       1021862        3.730985
6   Documentary        888427        3.515306
13      Mystery        846163        3.535035
2     Animation        604786        3.710527
9     Film-Noir        461872        3.578712
18      Western        456423        3.647356
17          War        380280        3.540412
11         IMAX         23435        3.843525


*So the* *top three genres are* **Drama, Comedy** *and* **Romance**.

**3. Find what genre movies have received the best and worst ratings based on user rating.**

**By Aveage Rating :**

In [35]:
# Sort genre_stats dataframe by 'Average_Rating' in descending order to get best ratings first
best_ratings_genres = genre_stats.sort_values(by='Average_Rating', ascending=False)

# Print the genre with the best ratings
print("Genre with the Best Average Rating:")
print(best_ratings_genres.head(1))

Genre with the Best Average Rating:
   Genre Total_Ratings  Average_Rating
11  IMAX         23435        3.843525


In [36]:
# Sort genre_stats dataframe by 'Average_Rating' in ascending order to get worst ratings first
worst_ratings_genres = genre_stats.sort_values(by='Average_Rating', ascending=True)

# Print the genre with the worst ratings
print("\nGenre with the Worst Average Rating:")
print(worst_ratings_genres.head(1))


Genre with the Worst Average Rating:
         Genre Total_Ratings  Average_Rating
6  Documentary        888427        3.515306


**By Total Rating :**

In [37]:
# Sort genre_stats dataframe by 'Average_Rating' in descending order to get best ratings first
best_ratings_genres = genre_stats.sort_values(by='Total_Ratings', ascending=False)

# Print the genre with the best ratings
print("Genre with Highest Number of Ratings:")
print(best_ratings_genres.head(1))

Genre with Highest Number of Ratings:
   Genre Total_Ratings  Average_Rating
7  Drama      11046346        3.584694


In [38]:
# Sort genre_stats dataframe by 'Average_Rating' in ascending order to get worst ratings first
worst_ratings_genres = genre_stats.sort_values(by='Total_Ratings', ascending=True)

# Print the genre with the worst ratings
print("\nGenre with Least Number of Ratings:")
print(worst_ratings_genres.head(1))


Genre with Least Number of Ratings:
   Genre Total_Ratings  Average_Rating
11  IMAX         23435        3.843525


**2. Create model that finds the best suited movie for on user in each genre.**

**Top 10 highest rated movies by user 1331154 :**

In [39]:
# Filter the dataset for the user with Cust_Id 1331154

user_df = df[df['Cust_Id'] == 1331154]

# Print some information to understand the data
print("Number of rows for user 1331154:", len(user_df))
print("Unique movies rated by user 1331154:", user_df['Movie_Id'].nunique())

# Split the 'Genre' column into separate genres
user_df['Genre'] = user_df['Genre'].str.split('|')

# Create a function to get top rated movies with genres and ratings
def get_top_rated_movies_with_genre_and_rating(df, top_n=10):
    # Sort by Rating in descending order
    top_rated_movies = df.sort_values(by='Rating', ascending=False)

    # Create a dataframe to store top rated movies with genres and ratings
    top_rated_movies_with_genre = pd.DataFrame(columns=['Movie_Id', 'Title', 'Genre', 'Rating'])

    for index, row in top_rated_movies.head(top_n).iterrows():
        movie_id = row['Movie_Id']
        title = row['Title']
        genres = row['Genre']
        rating = row['Rating']
        top_rated_movies_with_genre = top_rated_movies_with_genre.append({
            'Movie_Id': movie_id,
            'Title': title,
            'Genre': genres,
            'Rating': rating
        }, ignore_index=True)

    return top_rated_movies_with_genre

# Get top rated movies with genres and ratings for user 1331154
top_rated_movies_1331154 = get_top_rated_movies_with_genre_and_rating(user_df)

# Print top 10 rated movies with genres and ratings
print("Top 10 Rated Movies with Genres and Ratings for User with Cust_Id 1331154:")
print(top_rated_movies_1331154.head(10))

Number of rows for user 305344: 249
Unique movies rated by user 305344: 249
Top 10 Rated Movies with Genres and Ratings for User with Cust_Id 1331154:
  Movie_Id                      Title             Genre  Rating
0     1650   Washington Square (1997)           [Drama]     5.0
1     3379        On the Beach (1959)           [Drama]     5.0
2     1425    Fierce Creatures (1997)          [Comedy]     5.0
3      457       Fugitive, The (1993)        [Thriller]     5.0
4     3730   Conversation, The (1974)  [Drama, Mystery]     5.0
5     3650  Anguish (Angustia) (1987)          [Horror]     5.0
6     2128            Safe Men (1998)          [Comedy]     5.0
7      658     Billy's Holiday (1995)  [Drama, Musical]     5.0
8     2192         See the Sea (1997)        [Thriller]     5.0
9     2372        Fletch Lives (1989)          [Comedy]     5.0


**Top 10 movies recommended to user 1331154 :**

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# Taking a sample of 5 million rows due to memory constraints

# Batch size for processing
chunk_size = 500000

# Step 1: Importing the dataset
reader = pd.read_csv("/content/drive/My Drive/netflix_filtered.csv", chunksize=chunk_size)

# Initialize an empty DataFrame for the final result
final_df = pd.DataFrame()

# Function to process each chunk and concatenate into final_df
for chunk in reader:
    final_df = pd.concat([final_df, chunk], ignore_index=True)
    if len(final_df) >= 5000000:  # Stop after reading 5 million rows
        break

# Step 2: Splitting 'Genre' column into multiple binary columns
genres = final_df['Genre'].str.get_dummies('|')

# Combine final_df with genres
final_df = pd.concat([final_df, genres], axis=1)

# Step 3: Content-Based Filtering (based on genres)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(final_df['Genre'].apply(lambda x: ' '.join(x.split('|'))))

# Step 4: Collaborative Filtering (SVD)
scaler = StandardScaler()
final_df['Ratings_Normalized'] = scaler.fit_transform(final_df['Rating'].values.reshape(-1, 1))

user_item_matrix = final_df.pivot_table(index='Cust_Id', columns='Title', values='Ratings_Normalized').fillna(0)
matrix = user_item_matrix.values

svd = TruncatedSVD(n_components=10, random_state=42)
resultant_matrix = svd.fit_transform(matrix)

# Finding similar users
user_id = 1331154

# Directly locate the index for user ID
target_user_idx = final_df[final_df['Cust_Id'] == user_id].index[0]

target_user_vector = resultant_matrix[target_user_idx]

similar_users = []
for idx, vector in enumerate(resultant_matrix):
    if idx != target_user_idx:
        similarity_score = np.dot(target_user_vector, vector)
        similar_users.append((idx, similarity_score))

# Sorting similar users by similarity score
similar_users.sort(key=lambda x: x[1], reverse=True)


def hybrid_recommendation(user_id, num_recommendations=5):
    # Content-Based Recommendations
    genre_recommendations = content_based_recommendations(user_id, num_recommendations)

    # Collaborative Filtering Recommendations
    collaborative_recommendations = collaborative_filtering_recommendations(target_user_idx, similar_users, num_recommendations)

    # Combine both recommendations
    hybrid_recommendations = list(set(genre_recommendations + collaborative_recommendations))

    return hybrid_recommendations[:num_recommendations]

def content_based_recommendations(user_id, num_recommendations):
    target_user_genres = final_df[final_df['Cust_Id'] == user_id]['Genre'].values[0].split('|')
    target_user_movies = final_df[final_df['Cust_Id'] == user_id]['Title'].values
    similar_movies = []

    for genre in target_user_genres:
        if genre in final_df.columns:
            genre_movies = final_df[final_df[genre] == 1]['Title'].values
            for movie in genre_movies:
                if movie not in target_user_movies:
                    similar_movies.append((movie, genre))

    return similar_movies[:num_recommendations]

def collaborative_filtering_recommendations(target_user_idx, similar_users, num_recommendations):
    recommended_movies = []
    for user, _ in similar_users:
        user_movies = final_df[final_df.index == user]['Title'].values
        for movie in user_movies:
            if movie in user_item_matrix.columns and movie not in final_df[final_df.index == target_user_idx]['Title'].values:
                recommended_movies.append(movie)

    return recommended_movies[:num_recommendations]


# Get hybrid recommendations for user 1331154 with 10 recommendations
user_id = 1331154
hybrid_recs = hybrid_recommendation(user_id, num_recommendations=10)

# Create a DataFrame to store recommendations and genres
recommendations_df = pd.DataFrame(columns=['Movie', 'Genre'])

# Add recommendations and genres to the DataFrame
for idx, recommendation in enumerate(hybrid_recs, 1):
    if isinstance(recommendation, tuple):  # Check if the recommendation is in (movie, genre) format
        movie, genre = recommendation
        recommendations_df.loc[idx] = [movie, genre]
    else:
        movie = recommendation
        # Get genre for the movie
        genre = final_df[final_df['Title'] == movie]['Genre'].values[0]
        recommendations_df.loc[idx] = [movie, genre]

# Print the DataFrame
print(f"Hybrid Recommendations for user {user_id} :")
print(recommendations_df)


Hybrid Recommendations for user 1331154 :
                                Movie                       Genre
1                   Persuasion (1995)               Drama|Romance
2   Postman, The (Postino, Il) (1994)        Comedy|Drama|Romance
3                 Tom and Huck (1995)          Adventure|Children
4        Home for the Holidays (1995)                       Drama
5                    Toy Story (1995)                      Comedy
6      Kids of the Round Table (1995)  Adventure|Children|Fantasy
7                   Four Rooms (1995)                      Comedy
8                    Nico Icon (1995)                 Documentary
9             Wings of Courage (1995)      Adventure|Romance|IMAX
10                    Clueless (1995)              Comedy|Romance
